In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from requests.auth import HTTPBasicAuth

: 

In [ ]:
# URL del sitio de SharePoint
site_url = "https://grupoayesaa41015322-my.sharepoint.com/personal/ai_moreno_ibermatica_com/"

# Usuario y contraseña para la autenticación
username = "BIETOSIK"
password = "Iber.1859"

# Crear el objeto de autenticación
auth = HTTPBasicAuth(username, password)

# Crear el objeto de sitio SharePoint
site = Site(site_url, auth=auth)

# Conectar al sitio
site.auth.login()

# Ruta de la carpeta en SharePoint
folder_path = "/Documents/Q4Real/MB/Montabilidad_Pro"

# Obtener la lista de archivos en la carpeta
folder_files = site.get_folder(folder_path).files

<mark> *En este cuaderno de jupiter, se expone todo el código de preprocesado necesario para la detección de exóticos con tecnología cuántica.*

1. Se leen los ficheros .txt de los directorios  [test](https://grupoayesaa41015322-my.sharepoint.com/personal/ai_moreno_ibermatica_com/_layouts/15/onedrive.aspx?ct=1700555147845&or=OWA%2DNT&cid=5b628751%2D7368%2Db585%2D7884%2D814a07203a8f&fromShare=true&ga=1&id=%2Fpersonal%2Fai%5Fmoreno%5Fibermatica%5Fcom%2FDocuments%2FQ4Real%2FMB%2FMontabilidad%5FPro%2FDatos%5FTest) y [train](https://grupoayesaa41015322-my.sharepoint.com/personal/ai_moreno_ibermatica_com/_layouts/15/onedrive.aspx?ct=1700555147845&or=OWA%2DNT&cid=5b628751%2D7368%2Db585%2D7884%2D814a07203a8f&fromShare=true&ga=1&id=%2Fpersonal%2Fai%5Fmoreno%5Fibermatica%5Fcom%2FDocuments%2FQ4Real%2FMB%2FMontabilidad%5FPro%2FDatos%5FTrain) se genera un nuevo campo **"Set"** con el valor correspondiente; '*Test*' o '*Train*', para cada pedido del fichero. Se unen los dos ficheros.


In [ ]:


# Se leen los ficheros txt
test_csv = pd.read_csv('TV150GP_test.txt', delimiter='\t', names = ['Pedidos'])
train_csv = pd.read_csv('TV150GP_training.txt', delimiter='\t',names = ['Pedidos'])

#Se añade la columna 'Set'
test_csv['Set'] = 'Test'
train_csv['Set'] = 'Train'

dataset_trabajo = pd.concat([test_csv,train_csv])

2. Mediante la partición en subcadenas de cada linea del dataset de trabajo, se crean nuevas columnas  
   * Código -> 0 - 40
   * Expediente -> 44 - 58
   * Texto -> 55 - 1055

In [ ]:
dataset_trabajo['Codigo'] = dataset_trabajo['Pedidos'].str.slice(0, 40)  
dataset_trabajo['Expediente'] = dataset_trabajo['Pedidos'].str.slice(44, 58)   
dataset_trabajo['Texto'] = dataset_trabajo['Pedidos'].str.slice(55,1055)

3. Se vectoriza el campo texto en base al algortimo TF-IDF

In [ ]:
# Inicializar el vectorizador TF-IDF
vectorizador_tfidf = TfidfVectorizer()

# Aplicar el vectorizador TF-IDF a la columna 'Codigos'
matriz_tfidf = vectorizador_tfidf.fit_transform(dataset_trabajo['Texto'])

# Crear un DataFrame con los valores TF-IDF
nombres_columnas = vectorizador_tfidf.get_feature_names_out()
df_tfidf = pd.DataFrame(matriz_tfidf.toarray(), columns=list(map(str.upper, nombres_columnas)))

dataset_trabajo = dataset_trabajo.reset_index(drop=True)

# Concatenar el DataFrame original con el DataFrame TF-IDF
dataset_trabajo = pd.concat([dataset_trabajo, df_tfidf], axis=1)

4. Se parte en subcadenas la columna Codigo, se crean nuevas columnas  
   * Tipo_vehiculo -> 18-21 + 21 -22 + 24-25
   * N_Produccion -> 0-7
   * N_Pedido -> 8 -18
   * Ficha_Tecnica -> 32 - 40

In [ ]:
dataset_trabajo['Tipo_vehiculo'] = dataset_trabajo['Codigo'].str.slice(18, 21)  + '-'+ dataset_trabajo['Codigo'].str.slice(21, 22) + '-'+ dataset_trabajo['Codigo'].str.slice(24, 25)
dataset_trabajo['N_Produccion'] = dataset_trabajo['Codigo'].str.slice(0, 7)   
dataset_trabajo['N_Pedido'] = dataset_trabajo['Codigo'].str.slice(8,18)
dataset_trabajo['Ficha_Tecnica'] = dataset_trabajo['Codigo'].str.slice(32,40)

5. Se genera el atributo Chasis a partir de la columna Pedidos:
   * Chasis -> 26 - 32

In [ ]:
dataset_trabajo['Chasis'] = dataset_trabajo['Pedidos'].str.slice(26, 32)  

Se eliminan los codigos que no son necesarios indicados en el excel [Codigos a Excluir](https://grupoayesaa41015322-my.sharepoint.com/:x:/r/personal/ai_moreno_ibermatica_com/Documents/Q4Real/MB/Montabilidad_Pro/Datos_Train/Codigos%20a%20Excluir.xlsx?d=wf2433e0e674b4fda9944e88507bfeb63&csf=1&web=1&e=vuLREx)

In [ ]:
data_codigos_excluir = pd.read_excel('Codigos a Excluir.xlsx')

# Obtener la lista de columnas a eliminar
columnas_a_eliminar = data_codigos_excluir['Codigo'].tolist()

# Eliminar las columnas del primer DataFrame que coincidan con los códigos del segundo DataFrame
dataset_trabajo = dataset_trabajo.drop(columnas_a_eliminar, axis=1, errors='ignore')

Se agrupan los pedidos por categorias

In [ ]:
def categorizar_fila(fila):
    tipo_vehiculo = fila['Tipo_vehiculo']
    texto = fila['Texto']

    if "447-6" in tipo_vehiculo and "Z2K" in texto:
        return "BASE"
    elif "447-6" in tipo_vehiculo and "Z3K" in texto:
        return "PRO"
    elif "447-6" in tipo_vehiculo and "Z4K" in texto:
        return "SELECT"
    elif "447-7" in tipo_vehiculo and "ZK4" in texto:
        return "MIXTO"
    elif "447-7" in tipo_vehiculo and "ZK5" in texto:
        return "TOURIER"
    elif "447-7" in tipo_vehiculo and "ZQ8" in texto:
        return "TOURER PRO"
    elif "447-7" in tipo_vehiculo and "ZQ9" in texto:
        return "TOURER SELECT"
    elif "447-8" in tipo_vehiculo and "VQ9" in texto:
        return "CLASE V"
    elif "447-8" in tipo_vehiculo and "VR9" in texto:
        return "STYLE"
    elif "447-8" in tipo_vehiculo and "VQ1" in texto:
        return "AVANTGARDE"
    elif "447-8" in tipo_vehiculo and "VQ8" in texto:
        return "EXCLUSIVE"
    elif "447-8" in tipo_vehiculo and "ZE9" in texto:
        return "AVANTGARDE LINE ELECTRIC"
    elif "447-8" in tipo_vehiculo and "ZK7" in texto:
        return "MARCO POLO"
    else:
        return "NADA"

# Aplicar la función a cada fila del DataFrame y crear una nueva columna 'Categoria'
dataset_trabajo['Categoria'] = dataset_trabajo.apply(categorizar_fila, axis=1)

Del excel [Exóticos Validados](https://grupoayesaa41015322-my.sharepoint.com/:x:/r/personal/ai_moreno_ibermatica_com/Documents/Q4Real/MB/Montabilidad_Pro/Datos_Test/Exoticos_Validados.xlsx?d=w76955712c5734d20a67dcc0432fb8df6&csf=1&web=1&e=CqmQuH) se comprueba si en el dataset de trabajo hay algun pedido exotico no validado con la etiqueta 'Train', si la hay se le pone la etiqueta 'Test' ya que no ha sido validado aun.

In [ ]:
data_exoticos = pd.read_excel('Exoticos_Validados.xlsx')

# Modificar directamente la columna Set en dataset_trabajo
dataset_trabajo.loc[dataset_trabajo['N_Pedido'].isin(data_exoticos['ANR']), 'Set'] = 'Test'

Comparar con el fichero del 15 y actualizar chasis al pedido si no lo tiene.

In [ ]:
dataset_15 = pd.read_csv('TV150GP_ST15.txt', delimiter='\t', names = ['Pedidos'])

dataset_15['Chasis'] = dataset_15['Pedidos'].str.slice(26, 32)  
dataset_15['Codigo'] = dataset_15['Pedidos'].str.slice(0, 40)  
dataset_15['N_Pedido'] = dataset_15['Codigo'].str.slice(8,18)

# Actualizar los valores de la columna Chasis en dataset_trabajo
for index, row in dataset_trabajo.iterrows():
    n_pedido = row['N_Pedido']
    chasis_15 = dataset_15.loc[dataset_15['N_Pedido'] == n_pedido, 'Chasis']
    if not chasis_15.empty:
        # Verificar si la Serie no está vacía
        chasis_valor = chasis_15.iloc[0]
        dataset_trabajo.at[index, 'Chasis'] = chasis_valor

Exportar fichero de trabajo

In [ ]:
dataset_trabajo.to_csv('Dataset_Quantum.csv', index = False)

Se generan los .csv para Multiverse

In [ ]:
# Crear una nueva columna 'QuantumInspire_ID' como índice
dataset_trabajo['QuantumInspire_ID'] = range(1, len(dataset_trabajo) + 1)

# Crear un nuevo DataFrame con las columnas especificadas
df_quantum_ispire_previo = dataset_trabajo[['N_Pedido','Categoria', 'Set', 'Texto', 'Tipo_vehiculo', 'QuantumInspire_ID']].copy()

# Establecer 'QuantumInspire_ID' como índice
df_quantum_ispire_previo.set_index('QuantumInspire_ID', inplace=True)
df_quantum_ispire_anonimizado = df_quantum_ispire_previo[['Categoria', 'Set', 'Texto', 'Tipo_vehiculo']].copy()

# Exportar el DataFrame a un archivo CSV
df_quantum_ispire_previo.to_csv('Montabilidad_to_quantum_inspire_Previo.csv')
df_quantum_ispire_anonimizado.to_csv('Montabilidad_to_quantum_inspire_Anom.csv')